In [75]:
import matplotlib.pyplot as plt
import keras_ocr
import cv2
import math
import numpy as np

def midpoint(x1, y1, x2, y2):
    x_mid = int((x1 + x2)/2)
    y_mid = int((y1 + y2)/2)
    return (x_mid, y_mid)

pipeline = keras_ocr.pipeline.Pipeline()



Looking for /home/annamalaisenthil112/.keras-ocr/craft_mlt_25k.h5
Looking for /home/annamalaisenthil112/.keras-ocr/crnn_kurapan.h5


In [84]:
def inpaint_text(img_path, pipeline,s):
    # read image
    img = keras_ocr.tools.read(img_path)
    # generate (word, box) tuples 
    prediction_groups = pipeline.recognize([img])
    recognized_text = []
    # sorted_boxes = sorted(prediction_groups[0], key=lambda x: (x[1][0][1], midpoint(x[1][0][0], x[1][0][1], x[1][2][0], x[1][2][1]), x[1][0][0]))
    for box in prediction_groups[0]:
        recognized_text.append(box[0])
    mask = np.zeros(img.shape[:2], dtype="uint8")
    c=0
    for box in prediction_groups[0]:
        x0, y0 = box[1][0]
        x1, y1 = box[1][1] 
        x2, y2 = box[1][2]
        x3, y3 = box[1][3] 
        c+=1
        x_mid0, y_mid0 = midpoint(x1, y1, x2, y2)
        x_mid1, y_mi1 = midpoint(x0, y0, x3, y3)
        
        thickness = int(math.sqrt( (x2 - x1)**2 + (y2 - y1)**2 ))
        
        cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mi1), 255,    
        thickness)
        img = cv2.inpaint(img, mask, 7, cv2.INPAINT_NS)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mod_image_path="./data/img_copy/"+s+".png"
    with open ("image_text","a") as fh:
        fh.write(str(recognized_text)+"\n")
    cv2.imwrite(mod_image_path,img_rgb)
    return(img_rgb)

In [72]:
# path="./data/img/01245.png"
# img=inpaint_text(path,pipeline)

1/1 [==============================] - 1s 1s/step


In [54]:
# # Judging the sentiment in the text
# from textblob import TextBlob
# from textblob.sentiments import NaiveBayesAnalyzer,PatternAnalyzer 
# def tester(text):
#     return TextBlob(text,analyzer=PatternAnalyzer()).sentiment

In [56]:
# text="i am bad not"
# tester(text)


-0.6999999999999998


In [86]:
import json
with open ("image_text","w") as fh:
    pass
file_path = "./data/trial.jsonl"
c=0
with open(file_path, 'r') as file:
    for line in file:
        c+=1
        print(c)
        data_dict = json.loads(line)
        # print(data_dict)
        v=data_dict['id']
        image_path = "./data/img/"  # Replace with the actual path to your image
        s=str(v)
        if(len(s)==4):
            s='0'+s
        image_path+=s
        image_path+=".png"
        img=inpaint_text(image_path,pipeline,s)

1
1/1 [==============================] - 1s 648ms/step
2
1/1 [==============================] - 1s 1s/step
3
1/1 [==============================] - 0s 465ms/step
4
1/1 [==============================] - 1s 829ms/step
5
1/1 [==============================] - 1s 671ms/step


In [ ]:
#Run the previous code to judge if there is any change in detections